# REINFORCE

가장 간단한 Policy gradient algorithm

In [9]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [10]:
#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

In [11]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4, 128) # 4 inputs : state
        self.fc2 = nn.Linear(128, 2) # 2 outputs : action (left or right)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R # policy gradient
            loss.backward()
        self.optimizer.step() # update parameter
        self.data = []

In [12]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    
    for n_epi in range(2000):
        s, _ = env.reset()
        done = False
        
        while not done: # CartPole-v1 forced to terminates at 500 step.
            prob = pi(torch.from_numpy(s).float())
            m = Categorical(prob) # discrete probability distributions : allows you to sample from the distribution and compute various properties
            a = m.sample() # 하나의 action을 sampling
            s_prime, r, done, truncated, info = env.step(a.item())
            pi.put_data((r,prob[a])) # network update를 위해 reward와 해당 action의 확률을 저장
            s = s_prime
            score += r
            
        pi.train_net() # episode가 끝날 때마다 policy network를 update
        
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

In [13]:
if __name__ == '__main__':
    main()

# of episode :20, avg score : 22.9
# of episode :40, avg score : 21.7
# of episode :60, avg score : 25.65
# of episode :80, avg score : 28.55
# of episode :100, avg score : 23.5
# of episode :120, avg score : 27.7
# of episode :140, avg score : 26.95
# of episode :160, avg score : 27.35
# of episode :180, avg score : 24.35
# of episode :200, avg score : 33.9
# of episode :220, avg score : 31.4
# of episode :240, avg score : 30.4
# of episode :260, avg score : 28.6
# of episode :280, avg score : 36.0
# of episode :300, avg score : 36.1
# of episode :320, avg score : 38.4
# of episode :340, avg score : 40.1
# of episode :360, avg score : 35.05
# of episode :380, avg score : 48.1
# of episode :400, avg score : 57.3
# of episode :420, avg score : 55.25
# of episode :440, avg score : 29.95
# of episode :460, avg score : 38.7
# of episode :480, avg score : 39.5
# of episode :500, avg score : 52.8
# of episode :520, avg score : 43.05
# of episode :540, avg score : 49.65
# of episode :560, avg

# ActorCritic

In [14]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [15]:
#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98
n_rollout     = 10

In [2]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4,256)
        self.fc_pi = nn.Linear(256,2)
        self.fc_v = nn.Linear(256,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def pi(self, x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob
    
    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v
    
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, done_lst = [], [], [], [], []
        for transition in self.data:
            s,a,r,s_prime,done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r/100.0])
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0
            done_lst.append([done_mask])
        
        s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                                               torch.tensor(r_lst, dtype=torch.float), torch.tensor(s_prime_lst, dtype=torch.float), \
                                                               torch.tensor(done_lst, dtype=torch.float)
        self.data = []
        return s_batch, a_batch, r_batch, s_prime_batch, done_batch
  
    def train_net(self):
        s, a, r, s_prime, done = self.make_batch()
        td_target = r + gamma * self.v(s_prime) * done
        delta = td_target - self.v(s)
        
        pi = self.pi(s, softmax_dim=1)
        pi_a = pi.gather(1,a)
        loss = -torch.log(pi_a) * delta.detach() + F.smooth_l1_loss(self.v(s), td_target.detach())

        self.optimizer.zero_grad()
        loss.mean().backward()
        self.optimizer.step()         
      

/tmp/ipykernel_798452/1099603681.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \


# of episode :20, avg score : 20.9
# of episode :40, avg score : 27.2
# of episode :60, avg score : 24.9
# of episode :80, avg score : 34.2
# of episode :100, avg score : 30.5
# of episode :120, avg score : 39.0
# of episode :140, avg score : 39.9
# of episode :160, avg score : 67.6
# of episode :180, avg score : 75.9
# of episode :200, avg score : 64.7
# of episode :220, avg score : 94.9
# of episode :240, avg score : 121.7
# of episode :260, avg score : 120.7
# of episode :280, avg score : 176.2
# of episode :300, avg score : 176.1
# of episode :320, avg score : 221.6
# of episode :340, avg score : 186.7
# of episode :360, avg score : 269.6
# of episode :380, avg score : 276.8
# of episode :400, avg score : 197.3
# of episode :420, avg score : 228.5
# of episode :440, avg score : 292.2
# of episode :460, avg score : 273.9
# of episode :480, avg score : 223.6
# of episode :500, avg score : 320.1
# of episode :520, avg score : 226.4
# of episode :540, avg score : 288.6
# of episode :56

In [ ]:
def main():  
    env = gym.make('CartPole-v1')
    model = ActorCritic()    
    print_interval = 20
    score = 0.0

    for n_epi in range(5000):
        done = False
        s, _ = env.reset()
        while not done:
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, truncated, info = env.step(a)
                model.put_data((s,a,r,s_prime,done))
                
                s = s_prime
                score += r
                
                if done:
                    break                     
            
            model.train_net()
            
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

In [ ]:
if __name__ == '__main__':
    main()